In [2]:
from pathlib import Path
import pandas as pd
from prefect import flow, task
from prefect_gcp.cloud_storage import GcsBucket
import argparse
from prefect.tasks import task_input_hash
from datetime import timedelta
import os
import glob

In [61]:
path = 'C:\\Users\\Guilherme\\Documents\\WORKSPACE\\data-engineering-zoomcamp\\week3\\data' # use your path
all_files = glob.glob(os.path.join(path , "\\*.csv.gz"))

In [3]:
df = pd.concat(map(pd.read_csv, ['fhv_tripdata_2019-02.csv.gz', 'fhv_tripdata_2019-01.csv.gz','fhv_tripdata_2019-02.csv.gz','fhv_tripdata_2019-03.csv.gz','fhv_tripdata_2019-04.csv.gz','fhv_tripdata_2019-06.csv.gz','fhv_tripdata_2019-07.csv.gz','fhv_tripdata_2019-08.csv.gz','fhv_tripdata_2019-09.csv.gz','fhv_tripdata_2019-10.csv.gz','fhv_tripdata_2019-11.csv.gz','fhv_tripdata_2019-12.csv.gz']))

In [6]:
dataset_file = f"fhv_tripdata_2019-02"
url = f"https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/{dataset_file}.csv.gz"

In [7]:
df = pd.read_csv(url)

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1707649 entries, 0 to 1707648
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   dispatching_base_num    object 
 1   pickup_datetime         object 
 2   dropOff_datetime        object 
 3   PUlocationID            float64
 4   DOlocationID            float64
 5   SR_Flag                 float64
 6   Affiliated_base_number  object 
dtypes: float64(3), object(4)
memory usage: 91.2+ MB


In [15]:
df.head(10)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00037,2019-02-01 00:08:44,2019-02-01 00:23:35,264.0,265.0,NaN,B00037
1,B00037,2019-02-01 00:27:51,2019-02-01 00:32:54,264.0,265.0,NaN,B00037
2,B00037,2019-02-01 00:18:30,2019-02-01 00:25:45,264.0,265.0,NaN,B00037
3,B00037,2019-02-01 00:43:15,2019-02-01 00:48:29,264.0,265.0,NaN,B00037
4,B00037,2019-02-01 00:01:45,2019-02-01 00:09:13,264.0,265.0,NaN,B00037
5,B00037,2019-02-01 00:03:51,2019-02-01 00:16:36,264.0,265.0,NaN,B00037
6,B00037,2019-02-01 00:07:04,2019-02-01 00:13:47,264.0,265.0,NaN,B00037
7,B00037,2019-02-01 00:02:09,2019-02-01 00:07:32,264.0,265.0,NaN,B00037
8,B00095,2019-02-01 00:18:58,2019-02-01 00:25:52,264.0,265.0,NaN,B00095
9,B00095,2019-02-01 00:59:51,2019-02-01 01:06:07,264.0,265.0,NaN,B00095


In [25]:
df.shape

(1707649, 7)

In [45]:
df['PUlocationID'] = df['PUlocationID'].astype('Int64')

In [6]:
df['DOlocationID'].isnull().sum()

723690

In [7]:
mask = df['PUlocationID'].isnull() & df['DOlocationID'].isnull()
result = mask.sum()
result

717749

In [40]:
df[df['SR_Flag'].notna()]

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
18769,B03022,2019-02-01 08:32:09,2019-02-01 09:35:09,264.0,265.0,1.0,B03022
18770,B03022,2019-02-01 08:57:13,2019-02-01 09:14:36,264.0,265.0,2.0,B03022
23864,B03022,2019-02-01 09:17:34,2019-02-01 09:23:38,264.0,265.0,1.0,B03022
23865,B03022,2019-02-01 09:18:15,2019-02-01 09:39:25,264.0,265.0,2.0,B03022
23866,B03022,2019-02-01 09:00:12,2019-02-01 09:29:43,264.0,265.0,1.0,B03022
23867,B03022,2019-02-01 09:28:27,2019-02-01 09:29:43,264.0,265.0,2.0,B03022


In [37]:
df[df['PUlocationID'].isna()]

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
32565,B03059,2019-02-01 11:22:00,2020-02-01 11:40:00,NaN,NaN,NaN,B03059
